In [1]:
import matplotlib.pyplot as plt
import numpy as np
import Geodesics as gd
import Model as md
import cmath as cmt
import glob


In [2]:
p = 5; q = 4; nlayers = 5
nDecorr = 20
nMeasurements = 1000000
saveFreq = 10000

In [3]:
model = md.FractonModel(p,q,nlayers)

In [9]:

model = md.FractonModel(p,q,nlayers)
nBorder = len(model.border)
corr = np.zeros((saveFreq,int(len(model.border)/2)))
accum_spins = np.zeros((saveFreq,len(model.lattice)))

saveCounter = 0
for iteration in range(nMeasurements):
    for decorr in range(nDecorr):
        geod = np.random.choice(model.geodesicList)
        inner = np.random.randint(2)*2-1
        model.spins = model.spins*inner*geod.vect_inside()(model.centers)
    accum_spins[saveCounter,:] = model.spins
    corr[saveCounter,:] = model.getBorderCorrelations()/(nBorder)
    saveCounter += 1
    if(saveCounter == saveFreq):
        np.save("./States"+str(int(iteration/saveFreq))+".npy",np.array(accum_spins))
        np.save("./Correlations"+str(int(iteration/saveFreq))+".npy",corr)
        corr = np.zeros((saveFreq,int(len(model.border)/2)))
        accum_spins = np.zeros((saveFreq,len(model.lattice)))
        saveCounter = 0


In [34]:
corr_list = sorted(glob.glob("../Data/Correlations?.npy"))
corr_list += sorted(glob.glob("../Data/Correlations??.npy"))

states_list =sorted(glob.glob("../Data/States?.npy"))
states_list +=sorted(glob.glob("../Data/States??.npy"))
distances = np.linspace(1,280,280)

In [ ]:



corrs = np.zeros((nMeasurements,280))
for num,file in enumerate(corr_list):
    corr = np.load(file)
    corrs[10000*num:10000*(num+1),:] = corr
meanCorrelation = np.mean(corrs,axis = 0)
meanAbsCorrelation = np.mean(np.abs(corrs),axis = 0)
absMeanCorrelation = np.abs(meanCorrelation)
dmax = 150
plt.plot(distances[:dmax],meanCorrelation[:dmax],label = 'Mean')
#plt.plot(distances[:dmax],meanAbsCorrelation[:dmax], label = 'Mean Abs')
plt.plot(distances[:dmax],absMeanCorrelation[:dmax], label = 'Abs Mean')
plt.legend()

In [60]:
allCorrelations = np.zeros((560,280))

for num, file in enumerate(states_list):
    states = np.load(file)
    print(num)
    for idx,state in enumerate(states):
        model.spins = state
        allCorrelations[:,:] += model.getBorderCorrelationsEachSpin()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [61]:
np.save("../Data/MeansTest.npy",allCorrelations)